<a href="https://colab.research.google.com/github/aSafarpoor/OSN_FAD/blob/main/CLS/classifier_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#read data

In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
def load_txt_file(filename):
	with open(filename, 'r') as file:
		return [int(line.strip()) for line in file]

def load_txt_file_for_edges(filename):
	with open(filename, 'r') as file:
		return [list(map(int,line.strip().split())) for line in file]

In [147]:
def partitioner(x):
    midpoint = len(x) // 2

    # Split the list into two parts
    return x[:midpoint], x[midpoint:]

In [220]:
edges = load_txt_file_for_edges('edges.txt')


bknown = load_txt_file('btrain.txt')
sknown = load_txt_file('strain.txt')

b_known_in_test_phase,btrain = partitioner(bknown)
s_known_in_test_phase,strain = partitioner(sknown)


btest = load_txt_file('btest.txt')
stest = load_txt_file('stest.txt')
nodes = list(set(np.array(edges).reshape(-1)))
num_nodes = len(nodes)

print(len(bknown),len(sknown),len(btest),len(stest))

80 80 323 323


#Transductive

In [7]:
# !pip install torch torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv
!pip install torch
!pip install torch-geometric

In [130]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric.utils as pyg_utils
from torch_geometric.nn import GATConv
from sklearn.metrics import accuracy_score

from torch_geometric.utils import subgraph

In [172]:
class GCN_ETH(torch.nn.Module):
    def __init__(self, num_node_features, num_layers, hidden_width, dropout = True):
        super().__init__()

        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        input_width = num_node_features
        self.num_classes = 2
        for i in range(num_layers):
            if i == 0:
                self.convs.append(GCNConv(input_width, hidden_width, bias=False))
            elif i == num_layers - 1:
                self.convs.append(GCNConv(hidden_width, self.num_classes, bias=False))
            else:
                self.convs.append(GCNConv(hidden_width, hidden_width, bias=False))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        h = x
        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
            i += 1

        return F.log_softmax(h, dim=1)


In [169]:
class Edited_GCN_ETH(torch.nn.Module):
    def __init__(self, num_node_features, num_layers, hidden_width, dropout=True):
        super(Edited_GCN_ETH, self).__init__()

        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        input_width = num_node_features
        self.num_classes = 2  # Assuming binary classification

        for i in range(num_layers):
            if i == 0:  # First layer
                self.convs.append(GCNConv(input_width, hidden_width, bias=True))
            elif i == num_layers - 1:  # Last layer
                self.convs.append(GCNConv(hidden_width, self.num_classes, bias=True))
            else:  # Hidden layers
                self.convs.append(GCNConv(hidden_width, hidden_width, bias=True))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        h = x

        for i, conv in enumerate(self.convs):
            h = conv(h, edge_index)

            if i < len(self.convs) - 1:  # Apply activation and dropout for all but last layer
                h = F.relu(h)
                if self.dropout:
                    h = F.dropout(h, p=0.5, training=self.training)

        return F.log_softmax(h, dim=1)

In [161]:
class GAT_ETH(torch.nn.Module):
    def __init__(self, input_width, num_layers, hidden_width, num_classes, num_heads, dropout: bool = True):
        super().__init__()
        self.dropout = dropout
        self.num_classes = num_classes
        self.convs = torch.nn.ModuleList()

        for i in range(num_layers):
            if i == 0:  # First layer
                self.convs.append(GATConv(input_width, hidden_width, heads=num_heads))
            elif i == num_layers - 1:  # Last layer
                self.convs.append(GATConv(hidden_width * num_heads, self.num_classes, heads=1))
            else:  # Middle layers
                self.convs.append(GATConv(hidden_width * num_heads, hidden_width, heads=num_heads))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # x = self.conv1(x, edge_index)
        # # x = F.elu(x)
        # x = F.tanh(x)
        # x = self.conv2(x, edge_index)
        # h = x

        h = x
        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
                # h = F.elu(h)
            i += 1

        return F.log_softmax(h, dim=1)



In [105]:
class GCN1(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim, num_classes):
        super(GCN1, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [106]:
class GCN2(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim1, hidden_dim2, num_classes):
        super(GCN2, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = F.softmax(x,dim=1)
        return F.log_softmax(x, dim=1)


In [107]:
class GAT1(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim1, hidden_dim2, num_classes, num_heads=1):
        super(GAT1, self).__init__()
        self.gat1 = GATConv(num_node_features, hidden_dim1, heads=num_heads, concat=True)
        self.gat2 = GATConv(hidden_dim1 * num_heads, hidden_dim2, heads=num_heads, concat=True)
        self.gat3 = GATConv(hidden_dim2 * num_heads, num_classes, heads=1, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = F.relu(x)

        x = self.gat2(x, edge_index)
        x = F.relu(x)

        x = self.gat3(x, edge_index)

        return F.log_softmax(x, dim=1)



In [108]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(1, 16)  # 1 input feature, 16 output features
        self.conv2 = GCNConv(16, 2)  # 16 input features, 2 output features (binary classification)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x



In [149]:
def main_transductive(model):

    edge_index = torch.tensor([[e[0] for e in edges], [e[1] for e in edges]], dtype=torch.long)  # Replace with actual edges


    # Assign labels (0 for benign, 1 for sybil)
    labels = torch.tensor([0 if i in btrain or i in btest else 1 for i in range(num_nodes)])

    # Initialize the node features
    x = torch.zeros((num_nodes, 1))  # 1-dimensional embeddings, all zeros initially

    # Assign initial embeddings based on the conditions
    for node in btrain:
        x[node] = 0  # Benign training nodes
    for node in strain:
        x[node] = 1  # Sybil training nodes
    for node in btest + stest:
        x[node] = 0.5  # Test nodes

    # Create the PyTorch Geometric data object
    data_train = Data(x=x, edge_index=edge_index, y=labels)


    for node in b_known_in_test_phase:
        x[node] = 0  # Benign training nodes
    for node in s_known_in_test_phase:
        x[node] = 1


    data_test = Data(x=x, edge_index=edge_index, y=labels)


    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()





    # Split data into training and testing masks
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Assign masks for training and testing nodes
    train_mask[btrain + strain] = True
    test_mask[btest + stest] = True

    # Training loop
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(data_train)
        loss = loss_fn(out[train_mask], data_train.y[train_mask])  # Only consider training nodes for loss
        loss.backward()
        optimizer.step()
        return loss.item()

    # Testing function
    def test():
        model.eval()
        out = model(data_test)
        pred = out.argmax(dim=1)  # Get predictions
        train_acc = accuracy_score(data_test.y[train_mask].cpu(), pred[train_mask].cpu())
        test_acc = accuracy_score(data_test.y[test_mask].cpu(), pred[test_mask].cpu())
        return train_acc, test_acc

    # Training the model
    epochs = 100
    for epoch in range(epochs):
        loss = train()
        train_acc, test_acc = test()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

    # Final evaluation
    train_acc, test_acc = test()
    print(f'Final Train Accuracy: {train_acc:.4f}, Final Test Accuracy: {test_acc:.4f}')


In [150]:
main_transductive(GCN())

Epoch 0, Loss: 0.7117, Train Acc: 0.0750, Test Acc: 0.1223
Epoch 10, Loss: 0.6859, Train Acc: 0.8375, Test Acc: 0.7755
Epoch 20, Loss: 0.6720, Train Acc: 0.9000, Test Acc: 0.8452
Epoch 30, Loss: 0.6549, Train Acc: 0.9250, Test Acc: 0.8715
Epoch 40, Loss: 0.6320, Train Acc: 0.9125, Test Acc: 0.8653
Epoch 50, Loss: 0.5994, Train Acc: 0.9250, Test Acc: 0.8653
Epoch 60, Loss: 0.5558, Train Acc: 0.9375, Test Acc: 0.8700
Epoch 70, Loss: 0.5000, Train Acc: 0.9375, Test Acc: 0.8839
Epoch 80, Loss: 0.4301, Train Acc: 0.9375, Test Acc: 0.9009
Epoch 90, Loss: 0.3482, Train Acc: 0.9750, Test Acc: 0.9365
Final Train Accuracy: 0.9875, Final Test Accuracy: 0.9752


In [151]:
main_transductive(GCN1(num_node_features=1, hidden_dim=16, num_classes=2))

Epoch 0, Loss: 0.6918, Train Acc: 0.6000, Test Acc: 0.5248
Epoch 10, Loss: 0.6552, Train Acc: 0.9125, Test Acc: 0.8638
Epoch 20, Loss: 0.6207, Train Acc: 0.9125, Test Acc: 0.8622
Epoch 30, Loss: 0.5767, Train Acc: 0.9125, Test Acc: 0.8684
Epoch 40, Loss: 0.5241, Train Acc: 0.9250, Test Acc: 0.8731
Epoch 50, Loss: 0.4661, Train Acc: 0.9375, Test Acc: 0.8731
Epoch 60, Loss: 0.4068, Train Acc: 0.9375, Test Acc: 0.8824
Epoch 70, Loss: 0.3508, Train Acc: 0.9375, Test Acc: 0.8885
Epoch 80, Loss: 0.3011, Train Acc: 0.9375, Test Acc: 0.8932
Epoch 90, Loss: 0.2478, Train Acc: 0.9500, Test Acc: 0.9025
Final Train Accuracy: 0.9625, Final Test Accuracy: 0.9211


In [153]:
main_transductive(GCN2(num_node_features=1, hidden_dim1=16, hidden_dim2 = 16, num_classes=2))

Epoch 0, Loss: 0.6938, Train Acc: 0.5000, Test Acc: 0.5015
Epoch 10, Loss: 0.6812, Train Acc: 0.8875, Test Acc: 0.8483
Epoch 20, Loss: 0.6590, Train Acc: 0.9500, Test Acc: 0.9133
Epoch 30, Loss: 0.6121, Train Acc: 0.9625, Test Acc: 0.9226
Epoch 40, Loss: 0.5349, Train Acc: 0.9625, Test Acc: 0.9257
Epoch 50, Loss: 0.4472, Train Acc: 0.9625, Test Acc: 0.9180
Epoch 60, Loss: 0.3929, Train Acc: 0.9625, Test Acc: 0.9241
Epoch 70, Loss: 0.3708, Train Acc: 0.9625, Test Acc: 0.9257
Epoch 80, Loss: 0.3624, Train Acc: 0.9625, Test Acc: 0.9257
Epoch 90, Loss: 0.3589, Train Acc: 0.9625, Test Acc: 0.9241
Final Train Accuracy: 0.9625, Final Test Accuracy: 0.9241


In [167]:
main_transductive(GCN_ETH(num_node_features=1, num_layers=4, hidden_width=8))

Epoch 0, Loss: 0.7014, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 10, Loss: 0.6670, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 20, Loss: 0.6694, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 30, Loss: 0.6686, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 40, Loss: 0.6679, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 50, Loss: 0.6641, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 60, Loss: 0.6688, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 70, Loss: 0.6648, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 80, Loss: 0.6670, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 90, Loss: 0.6626, Train Acc: 0.5000, Test Acc: 0.5000
Final Train Accuracy: 0.5000, Final Test Accuracy: 0.5000


In [170]:
main_transductive(Edited_GCN_ETH(num_node_features=1, num_layers=3, hidden_width=16))

Epoch 0, Loss: 0.6925, Train Acc: 0.6500, Test Acc: 0.6517
Epoch 10, Loss: 0.6544, Train Acc: 0.9375, Test Acc: 0.8777
Epoch 20, Loss: 0.5520, Train Acc: 0.9125, Test Acc: 0.8731
Epoch 30, Loss: 0.4787, Train Acc: 0.9500, Test Acc: 0.8978
Epoch 40, Loss: 0.3201, Train Acc: 0.9625, Test Acc: 0.9164
Epoch 50, Loss: 0.2598, Train Acc: 0.9625, Test Acc: 0.9241
Epoch 60, Loss: 0.2638, Train Acc: 0.9625, Test Acc: 0.9180
Epoch 70, Loss: 0.2056, Train Acc: 0.9625, Test Acc: 0.9195
Epoch 80, Loss: 0.2126, Train Acc: 0.9625, Test Acc: 0.9241
Epoch 90, Loss: 0.2219, Train Acc: 0.9625, Test Acc: 0.9350
Final Train Accuracy: 0.9625, Final Test Accuracy: 0.9381


In [155]:
main_transductive(GAT1(num_node_features=1, hidden_dim1=16, hidden_dim2=16, num_classes=2, num_heads=8))

Epoch 0, Loss: 0.6893, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 10, Loss: 0.4014, Train Acc: 0.9375, Test Acc: 0.9149
Epoch 20, Loss: 0.1835, Train Acc: 0.9750, Test Acc: 0.9505
Epoch 30, Loss: 0.1021, Train Acc: 0.9750, Test Acc: 0.9582
Epoch 40, Loss: 0.0690, Train Acc: 0.9875, Test Acc: 0.9675
Epoch 50, Loss: 0.0335, Train Acc: 1.0000, Test Acc: 0.9737
Epoch 60, Loss: 0.0156, Train Acc: 1.0000, Test Acc: 0.9706
Epoch 70, Loss: 0.0086, Train Acc: 1.0000, Test Acc: 0.9721
Epoch 80, Loss: 0.0059, Train Acc: 1.0000, Test Acc: 0.9768
Epoch 90, Loss: 0.0046, Train Acc: 1.0000, Test Acc: 0.9752
Final Train Accuracy: 1.0000, Final Test Accuracy: 0.9752


In [156]:
main_transductive(GAT_ETH(input_width=1, num_layers=4, hidden_width=16, num_classes=2, num_heads=8, dropout= True))

Epoch 0, Loss: 0.6941, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 10, Loss: 0.9730, Train Acc: 0.6125, Test Acc: 0.5712
Epoch 20, Loss: 0.4364, Train Acc: 0.8000, Test Acc: 0.7136
Epoch 30, Loss: 0.2396, Train Acc: 0.9500, Test Acc: 0.9257
Epoch 40, Loss: 0.2189, Train Acc: 0.6375, Test Acc: 0.5851
Epoch 50, Loss: 0.5130, Train Acc: 0.9375, Test Acc: 0.8715
Epoch 60, Loss: 0.2382, Train Acc: 0.6625, Test Acc: 0.6223
Epoch 70, Loss: 0.3395, Train Acc: 0.8375, Test Acc: 0.7678
Epoch 80, Loss: 0.2723, Train Acc: 0.9500, Test Acc: 0.9241
Epoch 90, Loss: 0.2028, Train Acc: 0.9375, Test Acc: 0.9365
Final Train Accuracy: 0.7750, Final Test Accuracy: 0.7059


In [157]:
main_transductive(GAT1(num_node_features=1, hidden_dim1=4, hidden_dim2=4, num_classes=2, num_heads=8))

Epoch 0, Loss: 0.7000, Train Acc: 0.5000, Test Acc: 0.5000
Epoch 10, Loss: 0.6393, Train Acc: 0.9000, Test Acc: 0.8591
Epoch 20, Loss: 0.5146, Train Acc: 0.9375, Test Acc: 0.8901
Epoch 30, Loss: 0.3501, Train Acc: 0.9000, Test Acc: 0.8808
Epoch 40, Loss: 0.2544, Train Acc: 0.9375, Test Acc: 0.9102
Epoch 50, Loss: 0.2135, Train Acc: 0.9375, Test Acc: 0.9288
Epoch 60, Loss: 0.1774, Train Acc: 0.9500, Test Acc: 0.9489
Epoch 70, Loss: 0.1264, Train Acc: 0.9625, Test Acc: 0.9644
Epoch 80, Loss: 0.0448, Train Acc: 0.9750, Test Acc: 0.9783
Epoch 90, Loss: 0.0067, Train Acc: 1.0000, Test Acc: 0.9861
Final Train Accuracy: 1.0000, Final Test Accuracy: 0.9845


In [158]:
main_transductive(GCN2(num_node_features=1, hidden_dim1=6, hidden_dim2 = 6, num_classes=2))

Epoch 0, Loss: 0.6891, Train Acc: 0.6500, Test Acc: 0.6130
Epoch 10, Loss: 0.6824, Train Acc: 0.8250, Test Acc: 0.7910
Epoch 20, Loss: 0.6714, Train Acc: 0.8125, Test Acc: 0.7693
Epoch 30, Loss: 0.6544, Train Acc: 0.8500, Test Acc: 0.7786
Epoch 40, Loss: 0.6296, Train Acc: 0.8500, Test Acc: 0.7833
Epoch 50, Loss: 0.5944, Train Acc: 0.8500, Test Acc: 0.8019
Epoch 60, Loss: 0.5508, Train Acc: 0.8625, Test Acc: 0.8297
Epoch 70, Loss: 0.5097, Train Acc: 0.8750, Test Acc: 0.8452
Epoch 80, Loss: 0.4757, Train Acc: 0.8875, Test Acc: 0.8638
Epoch 90, Loss: 0.4482, Train Acc: 0.8875, Test Acc: 0.8715
Final Train Accuracy: 0.9125, Final Test Accuracy: 0.8808


#inductive

In [228]:
import random

def create_subgraph(data, nodes_subset):

    # Convert nodes_subset to tensor
    nodes_subset = torch.tensor(nodes_subset, dtype=torch.long)

    # Create a subgraph with only the edges between nodes in nodes_subset
    edge_index, edge_attr = subgraph(nodes_subset, data.edge_index, relabel_nodes=True)

    # Select the corresponding node features
    x = data.x[nodes_subset]

    # Select the corresponding labels (if any)
    y = data.y[nodes_subset] #if data.y is not None else None

    # Return the subgraph
    subgraph_data = Data(x=x, edge_index=edge_index, y=y)

    return subgraph_data


def main_inductive(model):

    edge_index = torch.tensor([[e[0] for e in edges], [e[1] for e in edges]], dtype=torch.long)  # Replace with actual edges

    # Define the set of unknown nodes (nodes not in btrain, strain, btest, stest)
    known_nodes = set(btrain + strain + btest + stest)
    unknown_nodes = list(set(range(num_nodes)) - known_nodes)

    # Sample 1/4 of unknown nodes for TRAIN
    random.shuffle(unknown_nodes)
    unknown_sample_for_train = unknown_nodes[:len(unknown_nodes)//4]
    print(len(unknown_sample_for_train),len(unknown_sample_for_train))
    # Create subgraph TRAIN consisting of btrain, strain, and 1/4 of unknown nodes
    train_nodes = btrain + strain + unknown_sample_for_train

    # Create subgraph TEST consisting of the remaining nodes
    test_nodes = list(set(nodes) - set(train_nodes))

    # Assign labels (0 for benign, 1 for sybil)
    labels = torch.tensor([0 if i in btrain
                           or i in btest
                           or i in  b_known_in_test_phase else 1 for i in range(num_nodes)])



    # Initialize the node features
    x = torch.zeros((num_nodes, 1))  # 1-dimensional embeddings, all zeros initially

    # Assign initial embeddings based on the conditions
    for node in nodes:
        x[node] = 0.5  # Test nodes
    for node in btrain:
        x[node] = 0  # Benign training nodes
    for node in strain:
        x[node] = 1  # Sybil training nodes


    # Create the PyTorch Geometric data object
    data_train = Data(x=x, edge_index=edge_index, y=labels)

    # Create subgraphs for training and testing
    train_subgraph = create_subgraph(data_train, train_nodes)

    xx = x[:]
    for node in b_known_in_test_phase:
        xx[node] = 0  # Benign training nodes
    for node in s_known_in_test_phase:
        xx[node] = 1

    data_test = Data(x=xx, edge_index=edge_index, y=labels)

    test_subgraph = create_subgraph(data_test, test_nodes)

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Training loop
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(train_subgraph)
        loss = loss_fn(out, train_subgraph.y)  # Only consider training nodes for loss
        loss.backward()
        optimizer.step()
        return loss.item()

    # Testing function
    def test():
        model.eval()
        out = model(test_subgraph)
        pred = out.argmax(dim=1)  # Get predictions
        test_acc = accuracy_score(test_subgraph.y.cpu(), pred.cpu())
        return test_acc

    # Training the model
    epochs = 100
    for epoch in range(epochs):
        loss = train()
        test_acc = test()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Acc: {test_acc:.4f}')

    # Final evaluation
    test_acc = test()
    print(f'Final Test Accuracy: {test_acc:.4f}')


    return train_subgraph,test_subgraph,x,xx


In [229]:
train_subgraph,test_subgraph,x,xx = main_inductive(GCN2(num_node_features=1, hidden_dim1=16, hidden_dim2 = 16, num_classes=2))

Epoch 0, Loss: 0.6512, Test Acc: 0.8969
Epoch 10, Loss: 0.4147, Test Acc: 0.8969
Epoch 20, Loss: 0.3668, Test Acc: 0.8969
Epoch 30, Loss: 0.3628, Test Acc: 0.8969
Epoch 40, Loss: 0.3623, Test Acc: 0.8969
Epoch 50, Loss: 0.3621, Test Acc: 0.8969
Epoch 60, Loss: 0.3621, Test Acc: 0.8969
Epoch 70, Loss: 0.3621, Test Acc: 0.8969
Epoch 80, Loss: 0.3621, Test Acc: 0.8969
Epoch 90, Loss: 0.3621, Test Acc: 0.8969
Final Test Accuracy: 0.8969


In [230]:
def temp(x):
    unique_elements, counts = torch.unique(x, return_counts=True)

    # Print unique elements and their frequency
    for element, count in zip(unique_elements, counts):
        print(f"Element: {element}, Frequency: {count}")

In [231]:
temp(train_subgraph.y)

Element: 0, Frequency: 49
Element: 1, Frequency: 960


In [232]:
temp(test_subgraph.y)

Element: 0, Frequency: 354
Element: 1, Frequency: 3079


In [233]:
temp(x)

Element: 0.0, Frequency: 80
Element: 0.5, Frequency: 4282
Element: 1.0, Frequency: 80


In [234]:
temp(xx)

Element: 0.0, Frequency: 80
Element: 0.5, Frequency: 4282
Element: 1.0, Frequency: 80


In [207]:
main_inductive(GAT1(num_node_features=1, hidden_dim1=4, hidden_dim2=4, num_classes=2, num_heads=8))

Epoch 0, Loss: 0.6352, Test Acc: 0.8978
Epoch 10, Loss: 0.2129, Test Acc: 0.8978
Epoch 20, Loss: 0.2131, Test Acc: 0.8978
Epoch 30, Loss: 0.2072, Test Acc: 0.8978
Epoch 40, Loss: 0.2041, Test Acc: 0.8978
Epoch 50, Loss: 0.2031, Test Acc: 0.8978
Epoch 60, Loss: 0.2030, Test Acc: 0.8978
Epoch 70, Loss: 0.2030, Test Acc: 0.8978
Epoch 80, Loss: 0.2030, Test Acc: 0.8978
Epoch 90, Loss: 0.2030, Test Acc: 0.8978
Final Test Accuracy: 0.8978


In [201]:
main_inductive(GAT_ETH(input_width=1, num_layers=4, hidden_width=16, num_classes=2, num_heads=8, dropout= True))

Epoch 0, Loss: 2.7685, Test Acc: 0.8980
Epoch 10, Loss: 0.2275, Test Acc: 0.8980
Epoch 20, Loss: 0.2392, Test Acc: 0.8980
Epoch 30, Loss: 0.2406, Test Acc: 0.8980
Epoch 40, Loss: 0.2116, Test Acc: 0.8980
Epoch 50, Loss: 0.2136, Test Acc: 0.8980
Epoch 60, Loss: 0.2157, Test Acc: 0.8980
Epoch 70, Loss: 0.2104, Test Acc: 0.8980
Epoch 80, Loss: 0.2064, Test Acc: 0.8980
Epoch 90, Loss: 0.2133, Test Acc: 0.8980


KeyboardInterrupt: 